In [6]:
import sys, os, time
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.xception import Xception, preprocess_input as pp_xception
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input as pp_vgg16
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input as pp_vgg19
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input as pp_inceptionv3
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input as pp_mobilenetv2
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input as pp_resnet50
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input as pp_resnet50v2
from tensorflow.keras.applications.resnet_v2 import ResNet101V2, preprocess_input as pp_resnet101v2
from tensorflow.keras.applications.resnet_v2 import ResNet152V2, preprocess_input as pp_resnet152v2
from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input as pp_densenet121
from tensorflow.keras.applications.densenet import DenseNet169, preprocess_input as pp_densenet169
from tensorflow.keras.applications.densenet import DenseNet201, preprocess_input as pp_densenet201
from tensorflow.keras.applications.nasnet import NASNetMobile, preprocess_input as pp_nasnetmobile
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input as pp_efficientnetb0
from keras import backend as K
import csv

In [7]:
models = [
    [Xception, 'xception', pp_xception],
    [VGG16, 'vgg16', pp_vgg16],
    [VGG19, 'vgg19', pp_vgg19],
    [InceptionV3, 'inceptionv3', pp_inceptionv3],
    [MobileNetV2, 'mobilenetv2', pp_mobilenetv2],
    [ResNet50V2, 'resnet50v2', pp_resnet50v2],
    [ResNet101V2, 'resnet101v2', pp_resnet101v2],
    [ResNet152V2, 'resnet152v2', pp_resnet152v2],
    [DenseNet121, 'densenet121', pp_densenet121],
    [DenseNet169, 'densenet169', pp_densenet169],
    [DenseNet201, 'densenet201', pp_densenet201],
    [NASNetMobile, 'nasnetmobile', pp_nasnetmobile],
    [EfficientNetB0, 'efficientnetb0', pp_efficientnetb0]
]

In [9]:
def infTime(x_val):
    start = time.time()
    y_pred = model.predict(x_val)
    end = time.time()
    timeTaken = (end - start)*1000
    return timeTaken

    K.clear_session()


In [10]:
inferenceTime = open("inferenceTime.csv", mode="w")
writer = csv.writer(inferenceTime) 
writer.writerow(["Test", 1, 2, 4, 16, 32, 64])

for i in range(13):

    gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
    session = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    session = tf.compat.v1.Session(config=config)
    os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"]="0"
    
    
    modelChosen = models[i][0]
    modelName = models[i][1]
    preprocess = models[i][2]

    fileName  = '.keras/models/' + modelName +'.h5'
    model = modelChosen(weights = fileName)

    if modelName not in ['xception', 'inceptionv3']:  # these models require 299 x 299
        x_val1 = np.load("data/x_val5.npy")

    else:
        x_val1 = np.load("data/x_val5_2.npy")

    toWrite = []    
    print(modelName)
    for batchSize in [1, 1, 2, 4, 16, 32, 64]:   # the first run always takes much longer, hence the first 1
        sum = 0
        for i in range(10):
            x_val = preprocess(x_val1[:batchSize])
            timeTaken = infTime(x_val)
            timePerPhoto = timeTaken/batchSize
            sum += timePerPhoto
        avg = "{:.2f}".format(sum/10)
        toWrite.append(avg)

    print(toWrite)
    writer.writerow(toWrite)

xception
['248.31', '26.75', '23.31', '15.20', '9.76', '10.16', '7.23']
vgg16
['68.65', '27.25', '33.73', '20.65', '9.25', '7.85', '4.15']
vgg19
['37.18', '30.02', '17.86', '11.83', '6.14', '5.14', '4.89']
inceptionv3
['138.63', '28.11', '40.58', '22.66', '9.02', '8.93', '3.97']
mobilenetv2
['71.49', '20.73', '16.82', '8.98', '3.37', '3.48', '1.54']
resnet50v2
['106.59', '39.40', '28.98', '16.73', '6.01', '6.04', '2.93']
resnet101v2
['130.80', '34.62', '17.82', '11.22', '5.78', '7.32', '4.74']
resnet152v2
['160.99', '36.83', '21.12', '13.67', '7.84', '10.54', '6.70']
densenet121
['168.33', '26.07', '43.15', '20.11', '7.40', '8.09', '3.00']
densenet169
['176.59', '27.41', '28.31', '15.40', '6.44', '8.56', '3.53']
densenet201
['202.32', '31.77', '28.53', '16.31', '7.41', '10.47', '4.41']
nasnetmobile
['229.65', '28.61', '19.81', '9.71', '3.84', '8.73', '2.17']
efficientnetb0
['140.29', '23.68', '19.27', '10.75', '4.11', '4.95', '2.20']
